In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!git clone 'https://github.com/danmohaha/CVPRW2019_Face_Artifacts.git'

Cloning into 'CVPRW2019_Face_Artifacts'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 106 (delta 2), reused 4 (delta 1), pack-reused 98
Receiving objects: 100% (106/106), 73.19 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [0]:
import sys
sys.path.insert(1,'CVPRW2019_Face_Artifacts/' )

In [8]:
test_dir = "drive/My Drive/DeepFake_test/"

test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4"])
len(test_videos)

5

In [0]:
import tensorflow as tf
from resolution_network import ResoNet
from solver import Solver
from easydict import EasyDict as edict
import cv2, yaml, os, dlib
from py_utils.vis import vis_im
import numpy as np
from py_utils.face_utils import lib
from py_utils.vid_utils import proc_vid as pv
import logging
import time

Load model weight and Face Detector weights.
Face Detector is **dlib**

In [5]:
cfg_file = 'CVPRW2019_Face_Artifacts/cfgs/res50.yml'
with open(cfg_file, 'r') as f:
    cfg = edict(yaml.load(f))
sample_num = 10

front_face_detector = dlib.get_frontal_face_detector()
lmark_predictor = dlib.shape_predictor('CVPRW2019_Face_Artifacts/dlib_model/shape_predictor_68_face_landmarks.dat')

tfconfig = tf.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth=True
# init session
sess = tf.Session(config=tfconfig)
# Build network
reso_net = ResoNet(cfg=cfg, is_train=False)
reso_net.build()
# Build solver
solver = Solver(sess=sess, cfg=cfg, net=reso_net)
solver.init()

W0211 18:59:52.154360 140396992898944 module_wrapper.py:139] From CVPRW2019_Face_Artifacts/resolution_network.py:36: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0211 18:59:52.171502 140396992898944 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1057: apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W0211 18:59:53.441703 140396992898944 module_wrapper.py:139] From CVPRW2019_Face_Artifacts/solver.py:35: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0211 18:59:53.442790 140396992898944 module_wrapper.py:139] From CVPRW2019_Face_Artifacts/solver.py:35: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0211 18:59:53.614640 140396992898944 module_wrapper.py:139] Fro

Loading checkpoint failed


In [0]:
def im_test(im):
    face_info = lib.align(im[:, :, (2,1,0)], front_face_detector, lmark_predictor)
    # Samples
    if len(face_info) == 0:
        logging.warning('No faces are detected.')
        prob = -1  # we ignore this case
    else:
        # Check how many faces in an image
        logging.info('{} faces are detected.'.format(len(face_info)))
        max_prob = -1
        # If one face is fake, the image is fake
        for _, point in face_info:
            rois = []
            for i in range(sample_num):
                roi, _ = lib.cut_head([im], point, i)
                rois.append(cv2.resize(roi[0], tuple(cfg.IMG_SIZE[:2])))
            vis_im(rois, 'tmp/vis.jpg')
            prob = solver.test(rois)
            prob = np.mean(np.sort(prob[:, 0])[np.round(sample_num / 2).astype(int):])
            if prob >= max_prob:
                max_prob = prob
        prob = max_prob
    return prob


def run(input_dir):
    logging.basicConfig(filename='run.log', filemode='w', format='[%(asctime)s - %(levelname)s] %(message)s',
                        level=logging.INFO)

    f_list = os.listdir(input_dir)
    prob_list = []
    for f_name in f_list:
        start_time = time.time()
        # Parse video
        f_path = os.path.join(input_dir, f_name)
        print('Testing: ' + f_path)
        logging.info('Testing: ' + f_path)
        suffix = f_path.split('.')[-1]
        if suffix.lower() in ['jpg', 'png', 'jpeg', 'bmp', 'tif', 'nef', 'raf']:
            im = cv2.imread(f_path)
            if im is None:
                prob = -1
            else:
                prob = im_test(im)

        elif suffix.lower() in ['mp4', 'avi', 'mov']:
            # Parse video
            imgs, frame_num, fps, width, height = pv.parse_vid(f_path)
            probs = []
            for fid, im in enumerate(imgs):
                logging.info('Frame ' + str(fid))
                prob = im_test(im)
                if prob == -1:
                    continue
                probs.append(prob)

            # Remove opt out frames
            if probs is []:
                prob = -1
            else:
                prob = np.mean(sorted(probs, reverse=True)[:int(frame_num / 3)])

        logging.info('Prob = ' + str(prob))
        prob_list.append(prob)
        print('Prob: ' + str(prob))
        elapsed = time.time() - start_time
        print("Elapsed %f sec" % (elapsed))

    sess.close()
    return prob_list

In [9]:
result = run(test_dir)

Testing: drive/My Drive/DeepFake_test/fwykevubzy.mp4


W0211 19:01:59.269298 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:01:59.924994 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:00.597832 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:00.903295 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:02.635726 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:03.294447 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:04.328392 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:07.140054 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:07.463052 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:07.794409 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:02:08.479954 140396992898944 <ipython-inp

Prob: 1.0
Elapsed 115.975063 sec
Testing: drive/My Drive/DeepFake_test/fsdrwikhge.mp4


W0211 19:03:44.365943 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:44.737117 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:45.060385 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:45.386825 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:45.725876 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:46.065707 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:46.405112 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:46.737993 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:47.067871 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:47.396714 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:03:47.726654 140396992898944 <ipython-inp

Prob: 1.0
Elapsed 106.512379 sec
Testing: drive/My Drive/DeepFake_test/aagfhgtpmv.mp4
Prob: 1.0
Elapsed 113.777638 sec
Testing: drive/My Drive/DeepFake_test/apatcsqejh.mp4


W0211 19:07:35.716698 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:36.029073 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:36.354912 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:36.678808 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:37.008107 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:38.291616 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:40.609908 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:44.364267 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:45.106264 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:45.433018 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:07:51.510833 140396992898944 <ipython-inp

Prob: 1.0
Elapsed 109.278668 sec
Testing: drive/My Drive/DeepFake_test/ciyoudyhly.mp4


W0211 19:10:49.400331 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:49.725434 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:51.494008 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:53.990834 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:54.319299 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:54.636724 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:54.956605 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:55.281687 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:55.586594 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:55.975730 140396992898944 <ipython-input-7-1e3c73af9f40>:5] No faces are detected.
W0211 19:10:56.385107 140396992898944 <ipython-inp

Prob: 1.0
Elapsed 113.077999 sec


In [10]:
result

[1.0, 1.0, 1.0, 1.0, 1.0]